# NER Histo vs NER Wiki

In [ ]:
!pip install tf2crf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/mhc/')

In [ ]:
%cd "/content/drive/My Drive/Colab Notebooks/mhc/"

In [1]:
import pandas as pd
import numpy as np

from model import BilstmCrf
from text import PreProcessing

import spacy
from spacy import displacy

from tqdm import tqdm
import multiprocessing as mp
from multiprocessing import Pool
from p_tqdm import p_map
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
nlp = spacy.load("en_core_web_md")

In [3]:
df = pd.read_csv("data/wiki/wiki.csv", sep="|")
df.Abstract = df.Abstract.parallel_apply(lambda x: PreProcessing.small_clean(x))
df.reset_index(drop=True, inplace=True)

In [4]:
def parse_data(data):
    sentences, labels, sentence, tag = [], [], [], []

    for text in data:
        doc = nlp(text)

        for sent in doc.sents:
            for word in sent:
                sentence.append(word.text)
                if word.ent_type_ is not "":
                    tag.append(word.ent_iob_+"-"+word.ent_type_)
                else:
                    tag.append(word.ent_iob_)
        
            sentences.append(" ".join(sentence))
            labels.append(" ".join(tag))

            sentence, tag = [], []
    return sentences, labels

In [5]:
X, y = parse_data(tqdm(df.Abstract.values))

100%|██████████| 1791/1791 [00:54<00:00, 32.74it/s]


In [ ]:
# We need to map the ents of spacy to histo